In [1]:
import torch
torch.cuda.is_available()

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
from datasets import load_dataset
from datasets import Dataset
dataset = load_dataset('trec')
#display(dataset)
dataset = dataset.remove_columns("label-fine")
#display(dataset)
dataset = dataset.rename_column("label-coarse", "label")
display(dataset)


train_validation_ds = dataset['train'].shuffle(seed=42).select([i for i in list(range(5000))])
train_ds = Dataset.from_dict(train_validation_ds[0:4500])
validation_ds = Dataset.from_dict(train_validation_ds[4500:5000])
test_ds = dataset['test'].shuffle(seed=42).select([i for i in list(range(500))])

display(train_ds)
display(validation_ds)
display(test_ds)


Downloading: 5.50kB [00:00, 2.80MB/s]                                                                                  
Using custom data configuration default


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]
Downloading:   0%|                                                                          | 0.00/336k [00:00<?, ?B/s]
Downloading:   2%|█▌                                                               | 8.19k/336k [00:00<00:10, 30.4kB/s]
Downloading:  12%|███████▉                                                         | 41.0k/336k [00:00<00:03, 82.5kB/s]
Downloading:  31%|████████████████████▊                                              | 104k/336k [00:00<00:01, 151kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████| 336k/336k [00:01<00:00, 305kB/s]
 50%|██████████████████████████████████████████                                          | 1/2 [00:03<00:03,  3.07s/it]
Downloading:   0%|                                                                         | 0.00/23.4k [00:00<?, ?B/s]
Downloading: 100%|██████████████████████

Dataset trec downloaded and prepared to C:\Users\DELL\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9. Subsequent calls will reuse this data.


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 403.36it/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 500
    })
})

Dataset({
    features: ['label', 'text'],
    num_rows: 4500
})

Dataset({
    features: ['label', 'text'],
    num_rows: 500
})

Dataset({
    features: ['label', 'text'],
    num_rows: 500
})

In [3]:
display(train_ds[0:5])

{'label': [0, 3, 2, 4, 1],
 'text': ['How can I transport files from one computer to another ?',
  'Who are the two sons of Ozzie and Harriet Nelson ?',
  'What does pH stand for ?',
  'How many people does Honda employ in the U.S. ?',
  "What newspaper returned a Pulitzer Prize for the fraudulent story Jimmy 's World ?"]}

In [4]:
display(validation_ds[0:5])

{'label': [1, 5, 4, 1, 1],
 'text': ['Which medium is Hamblen the first singing cowboy in ?',
  'What country was the setting of You Only Live Twice ?',
  'How many double-word-score spaces are there on a Scrabble Crossword Game board ?',
  'What is the nickname of the Cleveland Indians ?',
  'What drug is often used to treat AIDS patients ?']}

In [5]:
display(test_ds[0:5])

{'label': [4, 0, 1, 4, 1],
 'text': ['What is the population of Venezuela ?',
  'What does target heart rate mean ?',
  'What currency does Luxembourg use ?',
  'How long did Rip Van Winkle sleep ?',
  'Material called linen is made from what plant ?']}

In [6]:
from transformers import set_seed

set_seed(42)

In [7]:
#defining model:
from transformers import (AutoTokenizer, 
                          AutoModelForSequenceClassification, 
                          Trainer, EarlyStoppingCallback,
                          TrainingArguments)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenize_func = lambda sentences: tokenizer(sentences['text'], 
                                            padding='max_length', truncation=True, max_length = 50)



train_dataset = train_ds.map(tokenize_func, batched=True)
val_dataset = validation_ds.map(tokenize_func, batched=True)
test_dataset = test_ds.map(tokenize_func, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)

Parameter 'function'=<function <lambda> at 0x0000015FBED21E50> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 71.62ba/s]
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceCla

In [8]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels, average='macro')["f1"]
   return {"accuracy": accuracy, "f1": f1}


training_args = TrainingArguments(
    output_dir=r'C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\models\Original_model',          # output directory
    num_train_epochs=20,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
    
    logging_dir=r'C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\logs\Original_model',            # directory for storing logs
    save_total_limit = 1,
    load_best_model_at_end=True,
    #metric_for_best_model='f1',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
)


trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [9]:
#training:
trainer.train(ignore_keys_for_eval=['last_hidden_state', 'hidden_states', 'attentions'])
# We are going to get multiple loss values on each training step here


The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 4500
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 2
  Total optimization steps = 1400


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,1.779800,1.722508,0.308000,0.234870
1,1.565300,1.241647,0.656000,0.551855
2,0.913000,0.574167,0.832000,0.706153
3,0.436600,0.347246,0.910000,0.769917
4,0.270100,0.282225,0.912000,0.769772
5,0.175600,0.251100,0.932000,0.885070
6,0.120200,0.268482,0.926000,0.863137
7,0.099800,0.248575,0.936000,0.881303
8,0.056100,0.243750,0.934000,0.888903
9,0.038300,0.262992,0.932000,0.911562


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32
Using the latest cached version of the module from C:\Users\DELL\.cache\huggingface\modules\datasets_modules\metrics\accuracy\3e9ee15abf476145152fe4e9a9c1463ff95d3d65cdc555be9cfe061bdaeb1a14 (last modified on Sat Aug 13 21:10:49 2022) since it couldn't be found locally at accuracy, or remotely on the Hugging Face Hub.
Saving model checkpoint to C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\models\Original_model\checkpoint-70
Configuration saved in C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\models\Original_model\checkpoint-70\config.json
Model weights saved in C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\models\Original_model\checkpoint-70\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `Di

***** Running Evaluation *****
  Num examples = 500
  Batch size = 32
Saving model checkpoint to C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\models\Original_model\checkpoint-700
Configuration saved in C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\models\Original_model\checkpoint-700\config.json
Model weights saved in C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\models\Original_model\checkpoint-700\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32
Using the latest cached version of the module from C:\Users\DELL\.cache\huggingface\modules\datasets_modules\metrics\f1\6a64529c5745513ceb230ce9696115dcad6ae0fedad9946e3f2f723a65a0cd53 (last modified on Sat Aug 13 21:10:53 2022) since it couldn't be found locally at f1, or remotely on the Hugging Face Hub.
Saving model checkpoint to

TrainOutput(global_step=840, training_loss=0.45793447295824685, metrics={'train_runtime': 480.0114, 'train_samples_per_second': 187.496, 'train_steps_per_second': 2.917, 'total_flos': 698608378800000.0, 'train_loss': 0.45793447295824685, 'epoch': 11.99})

In [10]:
#save model:
trainer.save_state()
trainer.save_model()

Saving model checkpoint to C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\models\Original_model
Configuration saved in C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\models\Original_model\config.json
Model weights saved in C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\models\Original_model\pytorch_model.bin


In [11]:
#evaluate:
trainer.evaluate(test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32


{'eval_loss': 0.14332188665866852,
 'eval_accuracy': 0.97,
 'eval_f1': 0.9553362604455932,
 'eval_runtime': 43.3747,
 'eval_samples_per_second': 11.527,
 'eval_steps_per_second': 0.369,
 'epoch': 11.99}